In [2]:
import time

from optibook.synchronous_client import Exchange

kj/filesystem-disk-unix.c++:1734: warning: PWD environment variable doesn't match current directory; pwd = /


## Connecting to the Exchange

The following cell initiates a network connection between us and the exchange through which we can make requests to:

* Retrieve information about the state of the market, and
* Place orders.

In [3]:
exchange = Exchange()
exchange.connect()

2025-06-12 17:20:40,635 [asyncio   ] [MainThread  ] Using selector: EpollSelector
2025-06-12 17:20:40,639 [client    ] [Thread-1964 (_thread_entry_point)] synchronous_wrapper background thread started
2025-06-12 17:20:40,670 [client    ] [Thread-1964 (_thread_entry_point)] india-ff-2025-week2-1.optibook.net
2025-06-12 17:20:40,674 [client    ] [Thread-1964 (_thread_entry_point)] india-ff-2025-week2-1.optibook.net
2025-06-12 17:20:40,680 [client    ] [Thread-1964 (_thread_entry_point)] opened connection
2025-06-12 17:20:40,682 [client    ] [Thread-1964 (_thread_entry_point)] start read <StreamReader transport=<_SelectorSocketTransport fd=85 read=polling write=<idle, bufsize=0>>>
2025-06-12 17:20:40,687 [client    ] [Thread-1964 (_thread_entry_point)] logged in!


The following cell simply causes the Optibook system to produce fewer annoying messages like those above.

In [4]:
import logging

logging.getLogger('client').setLevel('ERROR')

# Implementing the Simple Trading Strategy

The following cell checks to see if it is possible to make a profit by buying _PHILIPS\_A_ and selling _PHILIPS\_B_. If so, it attempts to place two orders:

1. An _immediate-or-cancel_ order to purchase _PHILIPS\_A_, and
2. An _immediate-or-cancel_ order to sell _PHILIPS\_B_.

In [ ]:
VOLUME = 25


book_A = exchange.get_last_price_book('PHILIPS_A')
book_B = exchange.get_last_price_book('PHILIPS_B')

best_ask_A = book_A.asks[0].price
best_bid_B = book_B.bids[0].price

if best_bid_B > best_ask_A:
    exchange.insert_order('PHILIPS_A', price=best_ask_A, volume=VOLUME, side='bid', order_type='ioc')
    exchange.insert_order('PHILIPS_B', price=best_bid_B, volume=VOLUME, side='ask', order_type='ioc')
else:
    print("Purchasing A and selling B would not be profitable at this time.")

## What could go wrong?

What could go wrong with the code above?

In [9]:
VOLUME = 25


# get_last_price_book might return None or the bid and/or ask lists might be empty.
book_A = exchange.get_last_price_book('PHILIPS_A')
book_B = exchange.get_last_price_book('PHILIPS_B')

if book_A and book_A.asks and book_B and book_B.bids:

    best_ask_A = book_A.asks[0].price
    best_bid_B = book_B.bids[0].price

    if best_bid_B > best_ask_A:
        exchange.insert_order('PHILIPS_A', price=best_ask_A, volume=VOLUME, side='bid', order_type='ioc')
        exchange.insert_order('PHILIPS_B', price=best_bid_B, volume=VOLUME, side='ask', order_type='ioc')
    else:
        print("Purchasing A and selling B would not be profitable at this time.")
else:
    print("One or both of the order books are invalid")

Purchasing A and selling B would not be profitable at this time.


# Repeatedly Looking for Profitable Opportunities

If trading once is good, then trading often must be better! The following cell simply takes the logic above and applies it repeatedly in the hope of making more and more profit.

In [ ]:
VOLUME = 25


def carpe_diem():
    """See if there's a trading opportunity and if so, seize the day!"""
    book_A = exchange.get_last_price_book('PHILIPS_A')
    book_B = exchange.get_last_price_book('PHILIPS_B')

    if book_A is None or book_B is None:
        # One or both order books are invalid
        print("The received books are invalid")
        return

    # print("we here")

    if book_A.asks and book_B.bids and book_B.bids[0].price > book_A.asks[0].price:
        # print("we here")
        print(book_B.bids[0].price)
        print(book_A.asks[0].price)
        exchange.insert_order('PHILIPS_A', price=book_A.asks[0].price, volume=VOLUME, side='bid', order_type='ioc')
        exchange.insert_order('PHILIPS_B', price=book_B.bids[0].price, volume=VOLUME, side='ask', order_type='ioc')
        
    elif book_A.bids and book_B.asks and book_A.bids[0].price > book_B.asks[0].price:
        # print("we here")
        print(book_B.asks[0].price)
        print(book_A.bids[0].price)
        exchange.insert_order('PHILIPS_B', price=book_B.asks[0].price, volume=VOLUME, side='bid', order_type='ioc')
        exchange.insert_order('PHILIPS_A', price=book_A.bids[0].price, volume=VOLUME, side='ask', order_type='ioc')


# while True:
carpe_diem()
    # time.sleep(5.0)

## Final Algorithm

This is our final algorithm, combining - 

i) Pair trading arbitrage

ii) Exploiting the fact that Philips_B lags Philips_A

iii) Market making

iv) Aggressive rebalancing, if we go too long or short on either instrument (e.g. >100 or <-100)

In [ ]:
VOLUME = 25
MAX_POSITION = 200
MIN_POSITION = -200
REBALANCE_PRICE_TOLERANCE = 0.05
MAX_REBALANCE_VOLUME = 50

position_a = 0
position_b = 0

last_price_A = None

SPREAD_MIN = 0.05
SPREAD_MID = 0.20
VOLUME_MAX = 50
BASE_LOT_SIZE = 5

def volume_from_spread(spread):
    if spread < SPREAD_MIN:
        return 0
    elif spread < SPREAD_MID:
        vol = VOLUME_MAX * (spread - SPREAD_MIN) / (SPREAD_MID - SPREAD_MIN)
    else:
        vol = VOLUME_MAX
    return int(BASE_LOT_SIZE * round(vol / BASE_LOT_SIZE))

def carpe_diem():
    global position_a, position_b

    book_A = exchange.get_last_price_book('PHILIPS_A')
    book_B = exchange.get_last_price_book('PHILIPS_B')

    if book_A is None or book_B is None:
        return

    positions = exchange.get_positions()
    position_a = positions['PHILIPS_A']
    position_b = positions['PHILIPS_B']

    can_trade = decide_on_positions(position_a, position_b)
    last_trade_status = check_last_trade()
    missed_count = sum(not status for status in last_trade_status)

    # Volume calculations based on spreads
    spread_ab = (book_B.bids[0].price - book_A.asks[0].price) if (book_A.asks and book_B.bids) else 0
    spread_ba = (book_A.bids[0].price - book_B.asks[0].price) if (book_A.bids and book_B.asks) else 0

    volume_ab = volume_from_spread(spread_ab)
    volume_ba = volume_from_spread(spread_ba)

    if missed_count == 0 or missed_count == 2:
        if can_trade:
            if volume_ab > 0 and spread_ab > 0:
                insert_if_safe('PHILIPS_A', book_A.asks[0].price, volume_ab, 'bid', position_a)
                insert_if_safe('PHILIPS_B', book_B.bids[0].price, volume_ab, 'ask', position_b)
            elif volume_ba > 0 and spread_ba > 0:
                insert_if_safe('PHILIPS_B', book_B.asks[0].price, volume_ba, 'bid', position_b)
                insert_if_safe('PHILIPS_A', book_A.bids[0].price, volume_ba, 'ask', position_a)
        else:
            print("Out of position limits.")
    else:
        print("Previous trades may have failed or partially filled. Rebalancing...")
        rebalance_positions()

    exploit_lead_lag()
    market_make()

    time.sleep(0.1)
    print_trade_confirmations()


def insert_if_safe(instrument_id, price, volume, side, position):
    if (side == 'ask' and (position - volume) >= MIN_POSITION) or \
       (side == 'bid' and (position + volume) <= MAX_POSITION):
        exchange.insert_order(instrument_id, price=price, volume=volume, side=side, order_type='ioc')
    else:
        print(f"Skipped order for {instrument_id} due to position limits. Current pos: {position}")


def print_trade_confirmations():
    for instrument in ('PHILIPS_A', 'PHILIPS_B'):
        new_trades = exchange.poll_new_trades(instrument)
        total_traded = sum(t.volume for t in new_trades)

        for t in new_trades:
            direction = 'sold' if t.side == 'ask' else 'bought'
            print(f'Trade confirmation: {direction} {t.volume} lots of {instrument} at {t.price:.2f}')

        if total_traded == 0:
            print(f'Trade miss on {instrument}')


def decide_on_positions(pos_A, pos_B):
    return abs(pos_A) <= MAX_POSITION and abs(pos_B) <= MAX_POSITION


def check_last_trade():
    check_list = []
    for instrument in ['PHILIPS_A', 'PHILIPS_B']:
        new_trades = exchange.poll_new_trades(instrument)
        total_traded = sum(t.volume for t in new_trades)
        check_list.append(total_traded > 0)
    return check_list


def rebalance_positions():
    global position_a, position_b

    book_A = exchange.get_last_price_book('PHILIPS_A')
    book_B = exchange.get_last_price_book('PHILIPS_B')

    if not book_A or not book_B:
        return

    net = position_a + position_b
    volume = min(abs(net), MAX_REBALANCE_VOLUME)

    if net < 0:
        # Need to buy
        best_ask_A = book_A.asks[0].price if book_A.asks else float('inf')
        best_ask_B = book_B.asks[0].price if book_B.asks else float('inf')

        if best_ask_A <= best_ask_B:
            insert_if_safe('PHILIPS_A', best_ask_A + REBALANCE_PRICE_TOLERANCE, volume, 'bid', position_a)
        else:
            insert_if_safe('PHILIPS_B', best_ask_B + REBALANCE_PRICE_TOLERANCE, volume, 'bid', position_b)

    elif net > 0:
        # Need to sell
        best_bid_A = book_A.bids[0].price if book_A.bids else 0
        best_bid_B = book_B.bids[0].price if book_B.bids else 0

        if best_bid_A >= best_bid_B:
            insert_if_safe('PHILIPS_A', best_bid_A - REBALANCE_PRICE_TOLERANCE, volume, 'ask', position_a)
        else:
            insert_if_safe('PHILIPS_B', best_bid_B - REBALANCE_PRICE_TOLERANCE, volume, 'ask', position_b)


def exploit_lead_lag():
    global last_price_A, position_b

    book_A = exchange.get_last_price_book('PHILIPS_A')
    book_B = exchange.get_last_price_book('PHILIPS_B')

    if book_A is None or book_B is None:
        return

    mid_A = (book_A.asks[0].price + book_A.bids[0].price) / 2 if book_A.asks and book_A.bids else None
    mid_B = (book_B.asks[0].price + book_B.bids[0].price) / 2 if book_B.asks and book_B.bids else None

    if last_price_A is not None and mid_A is not None and mid_B is not None:
        move = abs(mid_A - last_price_A)
        vol_lag = volume_from_spread(move)

        if mid_A > last_price_A + SPREAD_MIN and (position_b + vol_lag <= MAX_POSITION):
            best_ask_B = book_B.asks[0].price
            insert_if_safe('PHILIPS_B', best_ask_B, vol_lag, 'bid', position_b)
            print(f"Lead-lag: A moved up, buying B at {best_ask_B:.2f}, volume {vol_lag}")

        elif mid_A < last_price_A - SPREAD_MIN and (position_b - vol_lag >= MIN_POSITION):
            best_bid_B = book_B.bids[0].price
            insert_if_safe('PHILIPS_B', best_bid_B, vol_lag, 'ask', position_b)
            print(f"Lead-lag: A moved down, selling B at {best_bid_B:.2f}, volume {vol_lag}")

    last_price_A = mid_A


def market_make():
    global position_a, position_b

    book_A = exchange.get_last_price_book('PHILIPS_A')
    book_B = exchange.get_last_price_book('PHILIPS_B')

    if not book_A or not book_B:
        return

    best_bid_A = book_A.bids[0].price if book_A.bids else 0
    best_ask_A = book_A.asks[0].price if book_A.asks else float('inf')

    best_bid_B = book_B.bids[0].price if book_B.bids else 0
    best_ask_B = book_B.asks[0].price if book_B.asks else float('inf')

    # Fixed volume market making
    if best_bid_A > best_bid_B and position_b + VOLUME <= MAX_POSITION:
        exchange.insert_order('PHILIPS_B', price=best_bid_A, volume=VOLUME, side='bid', order_type='ioc')
        print(f"Market making: placed bid on PHILIPS_B at {best_bid_A:.2f}")

    if best_bid_B > best_bid_A and position_a + VOLUME <= MAX_POSITION:
        exchange.insert_order('PHILIPS_A', price=best_bid_B, volume=VOLUME, side='bid', order_type='ioc')
        print(f"Market making: placed bid on PHILIPS_A at {best_bid_B:.2f}")

    if best_ask_A < best_ask_B and position_b - VOLUME >= MIN_POSITION:
        exchange.insert_order('PHILIPS_B', price=best_ask_A, volume=VOLUME, side='ask', order_type='ioc')
        print(f"Market making: placed ask on PHILIPS_B at {best_ask_A:.2f}")

    if best_ask_B < best_ask_A and position_a - VOLUME >= MIN_POSITION:
        exchange.insert_order('PHILIPS_A', price=best_ask_B, volume=VOLUME, side='ask', order_type='ioc')
        print(f"Market making: placed ask on PHILIPS_A at {best_ask_B:.2f}")


# Main loop
while True:
    carpe_diem()
    time.sleep(0.5)

Trade confirmation: bought 20 lots of PHILIPS_A at 83.90
Trade confirmation: sold 20 lots of PHILIPS_B at 84.40
Trade miss on PHILIPS_A
Trade miss on PHILIPS_B
Trade miss on PHILIPS_A
Trade miss on PHILIPS_B
Trade confirmation: bought 20 lots of PHILIPS_A at 83.40
Trade confirmation: sold 1 lots of PHILIPS_B at 84.20
Trade miss on PHILIPS_A
Trade miss on PHILIPS_B


KeyboardInterrupt: 